# Radial Velocity Measurement of LAMOST M Stars  

@author: Bo ZHANG (bozhang at bnu.edu.cn)  
@edit  : Jiadong LI
Please cite https://ui.adsabs.harvard.edu/abs/2021ApJS..256...14Z/abstract

In [1]:
import numpy as np
from astropy.io import fits
from slam.normalization import normalize_spectrum_iter, normalize_spectra_block
from preprocess import LAMOST

slam.normalization module will be deprecated in future


In [2]:
print("load Spec ...")
obs_dir = "data/"
wave_interp = np.load("models/wave_interp_R1800.npz")['wave']
obs_spec = LAMOST(root_dir=obs_dir, wave_resamp=wave_interp)


print("load RVM ...")
import joblib
from laspec.wavelength import air2vac, vac2air
rvm = joblib.load("models/R1800_rvm_Ca8500.dump") # 3. the path of the rvm file
rvm.wave_mod = vac2air(rvm.wave_mod)
rvm.cache_names = []



load Spec ...
load RVM ...


In [4]:
def measure_rv(spec, rvm):
    
    # cut spectrum  
    indcut = (spec['wave']>8000)&(spec['wave']<8950)
    wave_obs = spec['wave'][indcut]
    flux_obs = spec['flux'][indcut]
    flux_err = spec['invar'][indcut]**-0.5

    # normalize spectrum
    flux_norm, flux_cont =  normalize_spectrum_iter(wave_obs, flux_obs, niter=3)
    flux_norm_err = flux_err / flux_cont
    # measure rv
    rvr = rvm.measure(wave_obs, flux_norm, flux_norm_err, nmc=100)
    rvr["snr50"] = np.median(flux_obs/flux_err)
#     rvr["fp"] = fp
    rvr["obsid"] = spec["obsid"]
    rvr["rvteff"] = rvr["pmod"][0]
    rvr["rv16"],rvr["rv50"],rvr["rv84"] = rvr["rv_pct"]
    rvr["rv_unc"] = (rvr["rv84"]-rvr["rv16"])/2
    rvr["npix_obs"] = len(flux_obs)
    return rvr

rvr_list_valid = [measure_rv(obs_spec[_], rvm) for _ in range(len(obs_spec))]

In [5]:
from astropy.table import Table


trvr = Table(rvr_list_valid)
trvr.write("data/trvr.fits", overwrite=True) # 4. save to the file

In [6]:
trvr

rv_opt,rv_err,rv_best,ccfmax,success,imod,pmod [4],status,rv_pct [3],snr50,obsid,rvteff,rv16,rv50,rv84,rv_unc,npix_obs
float64,float64,float64,float64,bool,int64,float64,int64,float64,float32,int64,float64,float64,float64,float64,float64,int64
-4.708387091632844,16544.371720519193,-6.060606060606119,0.8596151446480795,True,417,4315.192430892548 .. -0.023213988195562862,0,-6.060606060606119 .. -2.5780612691294427,92.827255,1607211,4315.192430892548,-6.060606060606119,-4.6266145951482365,-2.5780612691294427,1.7412723957383383,488
3.2385883853782857,21114.187600654896,6.0606060606060055,0.8193599212316659,True,148,3629.499885287708 .. 0.0929780796377756,0,2.8012393926810777 .. 4.234912766158092,217.30652,1608025,3629.499885287708,2.8012393926810777,3.2708266794982617,4.234912766158092,0.7168366867385072,488
6.868070253289936,32068.47469567162,6.0606060606060055,0.6871246025008447,True,534,3610.999433267882 .. -0.06589334628795213,0,4.693868798103721 .. 10.730479459922462,71.88381,1608093,3610.999433267882,4.693868798103721,6.918557499420194,10.730479459922462,3.0183053309093704,488
